In [810]:
import pandas as pd
pd.set_option('max_column',100)
pd.set_option('max_row',100)

In [822]:
data_path = '../data/'
timeseries_path = f"{data_path}timeseries/"
ticker = 'MSFT'
pd.read_csv(f"{timeseries_path}{ticker}/Canonical Statement/Income Statement.csv",index_col=[0,1,2])

,,,2020-06-30,2020-03-31,2019-12-31,2019-09-30,2019-06-30,2019-03-31,2018-12-31,2018-09-30,2018-06-30,2018-03-31,2017-12-31,2017-09-30,2017-06-30,2017-03-31,2016-12-31,2016-09-30,2016-06-30,2016-03-31,2015-12-31,2015-09-30,2015-06-30,2015-03-31,2014-12-31,2014-09-30,2014-06-30,2014-03-31,2013-12-31,2013-09-30,2013-06-30,2013-03-31,2012-12-31,2012-09-30,2012-06-30,2012-03-31,2011-12-31,2011-09-30,2011-06-30,2011-03-31,2010-12-31,2010-09-30,2010-06-30,2010-03-31,2009-12-31,2009-09-30,2009-06-30,2009-03-31,2008-12-31,2008-09-30,2008-06-30,2008-03-31,2007-12-31,2007-09-30
filing_label,xbrl_tag,standard_label,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Weighted Average Number Of Diluted Shares Outstanding,us-gaap_weightedaveragenumberofdilutedsharesoutstanding,"Shares (diluted, weighted average)",-1.539300e+10,7.675000e+09,7.691000e+09,7.710000e+09,-1.552500e+10,7.744000e+09,7.768000e+09,7.766000e+09,-1.550900e+10,7.794000e+09,7.710000e+09,7.799000e+09,-1.568700e+10,7.813000e+09,7.830000e+09,7.876000e+09,-1.606600e+10,7.985000e+09,8.028000e+09,8.066000e+09,-1.663100e+10,8.237000e+09,8.297000e+09,8.351000e+09,-1.679700e+10,8.367000e+09,8.395000e+09,8.434000e+09,-1.689700e+10,8.429000e+09,8.444000e+09,8.494000e+09,NaN,8.498000e+09,NaN,8.490000e+09,-1.718200e+10,8.510000e+09,8.570000e+09,8.695000e+09,-1.788300e+10,8.876000e+09,8.951000e+09,8.983000e+09,8.996000e+09,8.904000e+09,8.914000e+09,9.183000e+09,9.470000e+09,NaN,NaN,NaN
Weighted Average Number Of Shares Outstanding Basic,us-gaap_weightedaveragenumberofsharesoutstandingbasic,"Shares (basic, weighted average)",-1.524700e+10,7.602000e+09,7.621000e+09,7.634000e+09,-1.536400e+10,7.672000e+09,7.692000e+09,7.673000e+09,-1.541600e+10,7.698000e+09,7.710000e+09,7.708000e+09,-1.552300e+10,7.725000e+09,7.755000e+09,7.789000e+09,-1.593000e+10,7.895000e+09,7.964000e+09,7.996000e+09,-1.646700e+10,8.167000e+09,8.228000e+09,8.249000e+09,-1.665000e+10,8.284000e+09,8.326000e+09,8.339000e+09,-1.677800e+10,8.364000e+09,8.393000e+09,8.396000e+09,NaN,8.401000e+09,NaN,8.392000e+09,-1.704100e+10,8.420000e+09,8.497000e+09,8.614000e+09,-1.772400e+10,8.767000e+09,8.856000e+09,8.914000e+09,8.945000e+09,8.891000e+09,8.903000e+09,9.084000e+09,9.328000e+09,NaN,NaN,NaN
Earnings Per Share Diluted,us-gaap_earningspersharediluted,EPS (diluted),1.460000e+00,1.400000e+00,1.510000e+00,1.380000e+00,1.710000e+00,1.140000e+00,1.080000e+00,1.140000e+00,1.140000e+00,9.500000e-01,-8.200000e-01,8.400000e-01,8.300000e-01,6.100000e-01,6.600000e-01,6.000000e-01,3.900000e-01,4.700000e-01,6.200000e-01,5.700000e-01,-4.000000e-01,6.100000e-01,7.100000e-01,5.400000e-01,5.500000e-01,6.800000e-01,7.800000e-01,6.200000e-01,5.900000e-01,7.200000e-01,7.600000e-01,5.300000e-01,-6.000000e-02,6.000000e-01,NaN,6.800000e-01,6.900000e-01,6.100000e-01,7.700000e-01,6.200000e-01,5.100000e-01,4.500000e-01,7.400000e-01,4.000000e-01,3.400000e-01,3.300000e-01,4.700000e-01,4.800000e-01,4.600000e-01,4.700000e-01,5.000000e-01,4.500000e-01
Earnings Per Share Basic,us-gaap_earningspersharebasic,EPS (basic),1.480000e+00,1.410000e+00,1.530000e+00,1.400000e+00,1.720000e+00,1.150000e+00,1.090000e+00,1.150000e+00,1.150000e+00,9.600000e-01,-8.200000e-01,8.500000e-01,8.400000e-01,6.200000e-01,6.700000e-01,6.000000e-01,4.000000e-01,4.800000e-01,6.300000e-01,5.800000e-01,-4.000000e-01,6.100000e-01,7.100000e-01,5.500000e-01,5.600000e-01,6.800000e-01,7.900000e-01,6.300000e-01,5.900000e-01,7.200000e-01,7.600000e-01,5.300000e-01,-6.000000e-02,6.100000e-01,NaN,6.800000e-01,7.000000e-01,6.200000e-01,7.800000e-01,6.300000e-01,5.200000e-01,4.600000e-01,7.500000e-01,4.000000e-01,3.400000e-01,3.300000e-01,4.700000e-01,4.800000e-01,4.600000e-01,4.700000e-01,5.000000e-01,4.600000e-01
Net Income Loss,us-gaap_netincomeloss,Net income,1.120200e+10,1.075200e+10,1.164900e+10,1.067800e+10,1.318700e+10,8.809000e+09,8.420000e+09,8.824000e+09,8.873000e+09,7.424000e+09,-6.302000e+09,6.576000e+09,6.513000e+09,4.801000e+09,5.200000e+09,4.690000e+09,3.122000e+09,3.756000e+09,4.

NameError: name 'timeseries_path' is not defined